In [1]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model(device='cuda')

model.translate_batch(['Счастливые лисички прыгают прыг скок', 'раз два три!'], 'ru', 'kaz')

['Счастливые лисички прыгают прыгают скак Бақытты лисички секіріп секіріп секіреді',
 'Бір, екі, үш рет!']

In [3]:
from scripts.helpers.dataset_manager import HFDataset
import pandas as pd

tt_ru = HFDataset('tat-ru', 'tatar-parallel-162k').get_df()

In [4]:
model._train(
    data=mhr_ru,
    src_lang='tat',
    dst_lang='rus',
    training_steps=100,
    save_every=20,
    ttl=10,
)

  0%|          | 0/100 [00:00<?, ?it/s]

0 1.693359375
Triggered time to leave! Current time: 1739916397.6928473. Start time: 1739916372.5352678. TTL: 10.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\serializat │
│ ion.py:619 in save                                                                               │
│                                                                                                  │
│    616 │                                                                                         │
│    617 │   if _use_new_zipfile_serialization:                                                    │
│    618 │   │   with _open_zipfile_writer(f) as opened_zipfile:                                   │
│ ❱  619 │   │   │   _save(obj, opened_zipfile, pickle_module, pickle_protocol, _disable_byteorde  │
│    620 │   │   │   return                                                                        │
│    621 │   else:                                                                                 │
│    622 │   │   with _open_file_like(f, 'wb') as opened_file:                                     │
│                                                                                                  │
│ C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\serializat │
│ ion.py:853 in _save                                                                              │
│                                                                                                  │
│    850 │   │   │   storage = storage.cpu()                                                       │
│    851 │   │   # Now that it is on the CPU we can directly copy it into the zip file             │
│    852 │   │   num_bytes = storage.nbytes()                                                      │
│ ❱  853 │   │   zip_file.write_record(name, storage.data_ptr(), num_bytes)                        │
│    854                                                                                           │
│    855                                                                                           │
│    856 def load(                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: [enforce fail at inline_container.cc:593] . PytorchStreamWriter failed writing file data/365: file 
write failed

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model._train(                                                                                │
│   2 │   data=tt_ru,                                                                              │
│   3 │   src_lang='tat',                                                                          │
│   4 │   dst_lang='rus',                                                                          │
│                                                                                                  │
│ C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\notebooks\..\scripts\helpers\model_manage │
│ r.py:172 in _train                                                                               │
│                                                                                                  │
│   169 │   │   │   │   self.tokenizer.save_pretrained(f"tokenizer.sft.{src_lang}_{dst_lang}.{i}   │
│   170 │   │                                                                                      │
│   171 │   │   # save final model                                                                 │
│ ❱ 172 │   │   self.model.save_pretrained(f"model.sft.{src_lang}_

In [3]:
import sys
sys.version

'3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]'